# Evaluating effects of FSL TOPUP and EPIC
..on individual MNI regions provided by Neuromorphometrics, Inc. (http://Neuromorphometrics.com/)
# Part 1: Running all corrections.

### __Procedure leading up to this assessment__:

The corrections output base directory in this toturial is:

In [3]:
corrections_base_directory = "../../epi_corrections_out_2019_04_25_372114315"

#### Nordic ICE (nICE) Batch Head Motion Correction (BHMC):
1. epi_corrections is run to create an EPI directory for nICE BHMC only. It will use ../scripts/EPI_hmc_nICE_prepare_dir.sh .

2. nICE BHMC is run on the prepared directory.

#### FSL TOPUP and EPIC intensity and distortion corrections:
3. epi_corrections is run completely, performing FSL TOPUP and EPIC corrections on the BHMC epi images. It will first use ../scripts/EPI_perfusion_anal_nICE_prepare_copy_back.sh to get the BHMC EPIs. After performing FSL TOPUP and EPIC corrections, respectively, it will perform FreeSurfer mri_robust_register to a structural image (FLAIR 3D) of EPI images, and assess similarities between combinations of corrected and non-corrected EPIs, using Normalized Mutual Information. Results should be in `<corrections_base_directory>`/TOPUP/topup_performance_metrics.txt and `<corrections_base_directory>`/EPIC/epic_performance_metrics.txt .

#### nICE Batch Perfusion Analysis (BPA) on uncorrected, TOPUP corrected and EPIC corrected EPIs:
4. ../scripts/EPI_perfusion_anal_nICE_make_uncorr_dir.sh is run to make a directory containing uncorrected EPIs of interest (thus containing the DSC EPIs only and not also the opposite phase encoded EPIs):

In [ ]:
!bash ../scripts/EPI_perfusion_anal_nICE_make_uncorr_dir.sh "../../epi_corrections_out_2019_04_25_372114315/EPI"

5. ../scripts/EPI_perfusion_anal_nICE_prepare_dir.sh is run three times to prepare directories for nICE BPA:

In [ ]:
!bash ../scripts/EPI_perfusion_anal_nICE_prepare_dir.sh "../../epi_corrections_out_2019_04_25_372114315/EPI_raw_DSC" raw
!bash ../scripts/EPI_perfusion_anal_nICE_prepare_dir.sh "../../epi_corrections_out_2019_04_25_372114315/EPI_applytopup" topup
!bash ../scripts/EPI_perfusion_anal_nICE_prepare_dir.sh "../../epi_corrections_out_2019_04_25_372114315/EPI_applyepic" epic

6. nICE BPA is run, see [](../scripts/nICE batch_perfusion_example.PNG "a screenshot of nICE BPA") for an example batch run.

7. Perfusion images are copied back to the respective directories:

In [ ]:
!bash ../scripts/EPI_perfusion_anal_nICE_prepare_copy_back.sh "../../epi_corrections_out_2019_04_25_372114315/EPI_raw_DSC"
!bash ../scripts/EPI_perfusion_anal_nICE_prepare_copy_back.sh "../../epi_corrections_out_2019_04_25_372114315/EPI_applytopup"
!bash ../scripts/EPI_perfusion_anal_nICE_prepare_copy_back.sh "../../epi_corrections_out_2019_04_25_372114315/EPI_applyepic"

#### Matlab SPM coregistration + MNI normalization
8. som_pipeline/coreg_norm_images.m is run to perform coregistration (to FLAIR 3D) (again, but now using SPM) and MNI normalization of raw, TOPUP and EPIC corrected EPIs, off-resonance fields (TOPUP), displacement fields (EPIC), as well as associated Normalized Relative CBV maps from BPA. Verify that epi_corrections_out_dir in ../spm_pipeline/coreg_norm_images.m is set to the correct directory then run:

In [ ]:
!matlab -nojvm -nodisplay -nosplash -nodesktop -r "run('../spm_pipeline/coreg_norm_images.m');exit;"

#### Calculate MNI CBV region histograms
9. Verify that epi_corrections_out_dir in spm_pipeline/compute_and_save_CBV_region_histograms.m is correct, then run:

In [ ]:
!matlab -nojvm -nodisplay -nosplash -nodesktop -r "run('../spm_pipeline/compute_and_save_CBV_region_histograms.m');exit;"

Matlab .mat files of MNI region histograms should now be saved to `<corrections_base_directory>`/CBV_histograms .